In [1]:
#注意、注意、注意:目前仅仅支持Sequential模型
#注意、注意、注意:目前仅仅支持Sequential模型
#注意、注意、注意:目前仅仅支持Sequential模型
import tensorflow as tf
from tensorflow import keras


In [2]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 归一化预处理数据，即将数据转换为（0，1）的范围.
train_images = train_images / 255.0
test_images = test_images / 255.0
print("data is ready!")

data is ready!


In [3]:
# 定义模型结构
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 12)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2028)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20290     
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 编译模型，SparseCategoricalCrossentropy是交叉熵函数，是标签是非one-hot编码下使用的
#如果标签是one-hot编码，需要使用CategoricalCrossentropy
model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
#训练模型
model.fit(train_images,train_labels,epochs=35,validation_split=0.1)
print("----------------------------------------------Train is ok!-----------------------------------------------")

Train on 54000 samples, validate on 6000 samples
Epoch 1/35
54000/54000 [==============================] - 8s 147us/sample - loss: 0.3075 - accuracy: 0.9116 - val_loss: 0.1323 - val_accuracy: 0.9657
Epoch 2/35
54000/54000 [==============================] - 6s 112us/sample - loss: 0.1291 - accuracy: 0.9628 - val_loss: 0.0947 - val_accuracy: 0.9743
Epoch 3/35
54000/54000 [==============================] - 7s 122us/sample - loss: 0.0917 - accuracy: 0.9737 - val_loss: 0.0795 - val_accuracy: 0.9780
Epoch 4/35
54000/54000 [==============================] - 7s 127us/sample - loss: 0.0746 - accuracy: 0.9784 - val_loss: 0.0690 - val_accuracy: 0.9803
Epoch 5/35
54000/54000 [==============================] - 6s 111us/sample - loss: 0.0648 - accuracy: 0.9809 - val_loss: 0.0626 - val_accuracy: 0.9820
Epoch 6/35
54000/54000 [==============================] - 6s 109us/sample - loss: 0.0561 - accuracy: 0.9838 - val_loss: 0.0627 - val_accuracy: 0.9827
Epoch 7/35
54000/54000 [===========================

In [5]:
model_result = model.evaluate(test_images,test_labels,verbose=0)
print(model_result)

[0.09461077652595849, 0.9793]


In [6]:
import tempfile
#保存未剪纸模型：
_, keras_file = tempfile.mkstemp('.h5',dir='./')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Saved baseline model to: D:\riky\jupyterpro\Tensorflow\tmpnxw9fphr.h5


In [7]:
import tensorflow_model_optimization as tfmot

import numpy as np
batch_size = 128
epochs = 2
validation_split = 0.1

num_images = train_images.shape[0] * (1 - validation_split)

end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
#PolynomialDecay方法定义一个具有多项式衰减功能的修剪计划，也就是说修剪过程中的稀疏度是变化的，网络参数逐渐减少，稀疏度逐渐提高。
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                        final_sparsity=0.80,
                                                        begin_step=0,
                                                        end_step=end_step)
#修改要在训练期间修剪的tf.keras层或模型，本例修剪的是整个模型的参数
model_for_pruning = prune_low_magnitude(model, pruning_schedule)
# 修剪后的模型需要重新编译
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

W0908 21:38:55.743342  9708 deprecation.py:323] From F:\Anaconda3\envs\tf2\lib\site-packages\tensorflow_model_optimization\python\core\sparsity\keras\pruning_wrapper.py:220: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


In [8]:
#训练和评估新的模型
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),  #回调函数，使其在训练过程中处理修减更新
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir), #提供用于跟踪进度和调试的日志
]
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 6s 110us/sample - loss: 0.2142 - accuracy: 0.9372 - val_loss: 0.4044 - val_accuracy: 0.8795
Epoch 2/2
54000/54000 [==============================] - 4s 77us/sample - loss: 0.2598 - accuracy: 0.9174 - val_loss: 0.1947 - val_accuracy: 0.9433


In [9]:
model_for_pruning_result = model_for_pruning.evaluate(test_images, test_labels, verbose=0)
print("Before:",model_result)
print("Now:",model_for_pruning_result)

Before: [0.09461077652595849, 0.9793]
Now: [0.20752001224756242, 0.9379]


In [10]:
#使用strip_pruning去除之前的不可训练权重,并保存模型

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)#使用strip_pruning去除之前的不可训练权重,并保存模型
_, pruned_keras_file = tempfile.mkstemp('.h5',dir='./')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: D:\riky\jupyterpro\Tensorflow\tmpxe8am329.h5


In [11]:
#使用gzip对比减枝前后文件大小
def get_gzipped_model_size(file):
    import os
    import zipfile
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    print('Saved pruned Keras model zip to:', zipped_file)
        
    return os.path.getsize(zipped_file)

In [12]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))

Saved pruned Keras model zip to: C:\Users\riky\AppData\Local\Temp\tmpjnr4j_0f.zip
Size of gzipped baseline Keras model: 78073.00 bytes
Saved pruned Keras model zip to: C:\Users\riky\AppData\Local\Temp\tmpcsj2kdq3.zip
Size of gzipped pruned Keras model: 25568.00 bytes
